## Complete experiment

In [ ]:
## imports

import numpy as np
import cupy as cp
import numpy.fft as fft
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.ndimage
import scipy.interpolate
from numpy.fft import fftshift, ifftshift, ifft2
import sigpy as sp
from sigpy.mri import dcf
from sigpy.mri.samp import radial
from tqdm.notebook import tqdm
import sigpy.plot as pl
import gc
import cupy as cp
from scipy.ndimage import map_coordinates
import cupy as cp
import numpy as np
from tqdm import tqdm


### 1. Create 2d Object

In [ ]:
def create_cylinder_mm(radius, matrix_size, height=0, fov=None):
    '''Create cylinder of radius, height centered at matrix_size'''
    Nz, Ny, Nx = matrix_size

    if fov==None:
        fov = max(2.5*2*radius, 1.5*height)

    ## Calculate voxel sizes based on FOV
    if height==0:
        voxel_size_z = 0
    else:
        voxel_size_z = fov / Nz

    voxel_size_x = fov / Nx
    voxel_size_y = fov / Ny
    print(f'voxel_size_z, voxel_size_y, voxel_size_x = [{voxel_size_z, voxel_size_y, voxel_size_x}]')

    ## Set up x, y ,z so that it goes from -Nx to +Nx (and respective Ny, Nz)
    x = (np.arange(Nx) - Nx//2) * voxel_size_x
    y = (np.arange(Ny) - Ny//2) * voxel_size_y 
    z = (np.arange(Nz) - Nz//2) * voxel_size_z 

    ## Create meshgrid, acts as image space ith voxels
    zz, yy, xx = np.meshgrid(z, y, x, indexing='ij')
    positions = np.vstack([zz.ravel(), yy.ravel(), xx.ravel()])
    
    ## distance from center in xy plane
    rr = np.sqrt(xx**2 + yy**2)
    print(rr.shape)

    cylinder = np.logical_and(rr <= radius, 
                              np.abs(zz) <= height//2 ).astype(float)
    
    fov = max(2.5*2*radius, 1.5*height)
    cylinder = np.squeeze(cylinder)
    return cylinder, fov, positions, voxel_size_x


def create_cylinder_with_objects_mm(radius, matrix_size, height=0, fov=None, add_objects=True):
    '''Create OG cylinder but add small objects for artifact visualization'''
    cylinder, fov, positions, voxel_size_x = create_cylinder_mm(radius, matrix_size, height, fov)
    
    if add_objects:
        Nz, Ny, Nx = matrix_size
        if height == 0:
            voxel_size_z = 0
        else:
            voxel_size_z = fov / Nz
        voxel_size_x = fov / Nx
        voxel_size_y = fov / Ny
        

        x = (np.arange(Nx) - Nx//2) * voxel_size_x
        y = (np.arange(Ny) - Ny//2) * voxel_size_y
        z = (np.arange(Nz) - Nz//2) * voxel_size_z
        zz, yy, xx = np.meshgrid(z, y, x, indexing='ij')
        
        # Array to store objects
        objects = []
        
        # Small circle upper right
        obj1_center = (radius * 0.5, radius * 0.5)
        obj1_radius = radius * 0.1
        obj1_mask = (xx - obj1_center[0])**2 + (yy - obj1_center[1])**2 <= obj1_radius**2
        objects.append(('circle1', obj1_mask, 0.7))  # Intensity 0.7
        
        # Small rectangle near center
        obj3_width = radius * 0.15
        obj3_height = radius * 0.05
        obj3_center = (radius * 0.1, -radius * 0.1)
        obj3_mask = (np.abs(xx - obj3_center[0]) <= obj3_width/2) & (np.abs(yy - obj3_center[1]) <= obj3_height/2)
        objects.append(('rectangle', obj3_mask, 0.5))  # Set intensity = 0.5
        
        # Thin line 
        obj4_mask = (np.abs(xx + radius * 0.2) <= radius * 0.02) & (np.abs(yy) <= radius * 0.3)
        objects.append(('line', obj4_mask, 1.5))  # Set intensity 1.5
        
        # add objects where they exist and cylinder is nonzero, else just return cylinder
        for obj_name, obj_mask, intensity in objects:
            cylinder = np.where(obj_mask & (cylinder > 0), intensity, cylinder)
    
    cylinder = np.squeeze(cylinder)
    return cylinder, fov, positions, voxel_size_x


def show_2d_cylinder(c):
    plt.figure(1)
    plt.imshow(c, cmap='gray')
    plt.title("2D Image of 2d Cylinder")
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.colorbar(label="Intensity")
    plt.savefig('og_2d_cylinder.png')
    plt.show()




### 2. Create Radial Trajectory

In [ ]:
def center_out_GA_radial(ntr, nro, ndims, img_size, fov_orig):
    '''Create center-out golden-angle radial trajectory using Sigpy
    
    Inputs
    ----------------------
    ntr (int): Number of repetitions = number of spokes collected
    nro (int) : Number of readout points = number of ksp samples per spoke
    ndims (int): Number of kspace dimensions
    img_size (tuple) : (Nx, Ny) size of image
      
    '''
    coord_shape = (ntr, nro, ndims)
    if ndims != len(img_size):
        Warning("K-space dimensions not equal to image dimensions")
    else:
        radial_coords_pixel = radial(coord_shape, img_size, golden=True)
        radial_coords_physical = radial_coords_pixel/fov_orig
        return radial_coords_pixel, radial_coords_physical


def plot_GA_trajectory(radial_coords_pixel):
    num_spokes = radial_coords_pixel.shape[0]
    for i in range(num_spokes):
        plt.plot(radial_coords_pixel[i,:,0], radial_coords_pixel[i,:,1])
    plt.title(f"Center-out Radial Trajectory with {num_spokes} spokes")
    plt.savefig(f'radial_coords_{num_spokes}sp.png')
    plt.show()

### 3. Timing + Motion Information

In [ ]:
def get_acq_times(num_spokes, total_scan_time=1.0):
    '''Assign time point to each spoke'''
    acq_times = np.linspace(0, total_scan_time, num_spokes)
    return acq_times


def sinusoidal_motion(t, amplitude, frequency, direction='x'):
    '''Calculate position offset due to sinusoidal motion
    
    Inputs
    ------------------------
    t : time points (s)
    amplitude: motion amplitude (mm)
    frequency: motion frequency (Hz)
    direction: 'x' , 'y', 'xy'
    '''

    displacement = amplitude*np.sin(2*np.pi*frequency*t)

    if direction=='x':
        return displacement, 0
    elif direction=='y':
        return 0, displacement
    elif direction=='xy':
        return displacement, displacement
    else:
        return 0,0
    

### 3.5. Plot Motion over Time

In [ ]:
def plot_motion_over_time(ntr, motion_params, num_point=1000):
    acq_times = get_acq_times(ntr, motion_params['scan_time'])
    # print(f'acq_times = {acq_times}')
    dx_array, dy_array = [], []

    for t in acq_times:
        dx, dy = sinusoidal_motion(t, motion_params['amplitude'], motion_params['frequency'], motion_params['direction'])
        dx_array.append(dx)
        dy_array.append(dy)

    dx_array = np.array(dx_array)
    dy_array = np.array(dy_array)
    # print(f'dx array = {dx_array}')
    # print(f'dy array = {dy_array}')

    plt.figure(1)
    plt.plot(acq_times, dx_array, 'b-', linewidth=2, label="X displacement")
    plt.plot(acq_times, dy_array, 'r-', linewidth=2, label="Y displacement")
    plt.xlabel("Time (s)")
    plt.ylabel("Displacement (mm)")
    plt.legend()
    plt.grid(True)
    plt.title("Phantom Dynamics: Motion vs Time")
    plt.savefig(f"motion_v_time_amp{motion_params['amplitude']}_freq{motion_params['frequency']}_dir{motion_params['direction']}.png")

### 4. Fourier Encoding of object

In [ ]:
import cupy as cp
import numpy as np
from tqdm import tqdm


def fourier_encoding_radial_motion_gpu(radial_coords, cylinder_orig, fov_orig, 
                                     upsamp, orig_size, motion_params=None):
    # Move data to GPU
    kx_coords_gpu = cp.asarray(radial_coords[..., 1])
    ky_coords_gpu = cp.asarray(radial_coords[..., 0])
    
    num_spokes = radial_coords.shape[0]
    num_points = radial_coords.shape[1]
    grid_size = int(upsamp * orig_size)
    
    x_pos_base = cp.linspace(-fov_orig/2, fov_orig/2, grid_size)
    y_pos_base = cp.linspace(-fov_orig/2, fov_orig/2, grid_size)
    
    # Move cylinder to GPU
    pad_size = (grid_size - orig_size) // 2
    cylinder_padded_cpu = np.pad(cylinder_orig, 
                                ((pad_size, pad_size), (pad_size, pad_size)),
                                mode='constant', constant_values=0)
    cylinder_padded_gpu = cp.asarray(cylinder_padded_cpu)
    
    S_gpu = cp.zeros((num_spokes, num_points), dtype=cp.complex64)

    if motion_params is not None:
        acq_times = get_acq_times(num_spokes, motion_params['scan_time'])
    
    # Process spoke by spoke
    for spoke in tqdm(range(num_spokes), desc='Processing spokes (with motion) on GPU'):
        kx_spoke = kx_coords_gpu[spoke, :]
        ky_spoke = ky_coords_gpu[spoke, :]

        ## Every time we acquire a spoke, assume object has been affected by motion
        if motion_params is not None:
            t = acq_times[spoke]
            dx, dy = sinusoidal_motion(t, motion_params['amplitude'],
                                       motion_params['frequency'],
                                       motion_params['direction'])
            
            ## Shift coordinates by motion
            x_pos = x_pos_base + dx
            y_pos =  y_pos_base + dy
        else:
            x_pos = x_pos_base
            y_pos = y_pos_base

        X_gpu, Y_gpu = cp.meshgrid(x_pos, y_pos)
        # Compute all encoding patterns for this spoke on GPU
        F_all = cp.exp(-1j * 2 * cp.pi * (
            kx_spoke[:, None, None] * X_gpu[None, :, :] + 
            ky_spoke[:, None, None] * Y_gpu[None, :, :]
        ))
        
        # Compute inner products on GPU
        S_gpu[spoke, :] = cp.sum(cylinder_padded_gpu[None, :, :] * F_all, axis=(1, 2))
    
    # Move result back to CPU
    S = cp.asnumpy(S_gpu)
    return S



def fourier_encoding_cartesian_motion_gpu(matrix_size, cylinder_orig, fov_orig, upsamp, orig_size, motion_params=None):
    # Move data to GPU
    grid_size = int(upsamp * orig_size)
    S_gpu = cp.zeros((grid_size, grid_size), dtype=cp.complex64)
    Nx, Ny = grid_size, grid_size

    dkx = 1/fov_orig
    dky = 1/fov_orig
    
    x_pos = cp.linspace(-fov_orig/2, fov_orig/2, grid_size)
    y_pos= cp.linspace(-fov_orig/2, fov_orig/2, grid_size)
    X_gpu, Y_gpu = cp.meshgrid(x_pos, y_pos, indexing='ij')
    
    # Move cylinder to GPU
    pad_size = (grid_size - orig_size) // 2
    cylinder_padded_cpu = np.pad(cylinder_orig, 
                                ((pad_size, pad_size), (pad_size, pad_size)),
                                mode='constant', constant_values=0)
    cylinder_padded_gpu = cp.asarray(cylinder_padded_cpu)

    if motion_params is not None:
        acq_times = get_acq_times(Ny, motion_params['scan_time'])
    
    
    # Process line by line
    for ky_idx in range(0, Ny):


        if motion_params is not None:
            t = acq_times[ky_idx]
            dx, dy = sinusoidal_motion(t, motion_params['amplitude'], motion_params['frequency'], motion_params['direction'])

        else:
            dx, dy = 0, 0

        x_pos = cp.linspace(-fov_orig/2, fov_orig/2, grid_size) + dx
        y_pos= cp.linspace(-fov_orig/2, fov_orig/2, grid_size) + dy
        X, Y = cp.meshgrid(x_pos, y_pos, indexing='ij')

        n_ky = (ky_idx - Ny//2) * dky
    
        for kx_idx in range(0, Nx):
            n_kx = (kx_idx - Nx//2) * dkx

            # Compute all encoding patterns for this pt on GPU
            F_all = cp.exp(-1j * 2 * cp.pi * (
                n_kx * X[None, :, :] + 
                n_ky * Y[None, :, :]
            ))
            
            # Compute inner products on GPU
            S_gpu[kx_idx, ky_idx] = cp.sum(cylinder_padded_gpu[None, :, :] * F_all, axis=(1, 2))
    
    # Move result back to CPU
    S = cp.asnumpy(S_gpu)
    return S

### 5. NUFFT Reconstruction

In [ ]:
def analytical_density_weights(radial_coords):
    """Analytical density compensation for radial sampling"""
    kx = radial_coords[..., 0]
    ky = radial_coords[..., 1]
    kr = np.sqrt(kx**2 + ky**2)
    
    eps = 1e-8

    weights = kr+eps
    
    return weights


def reconstruct_from_2d_radial(radial_coords, radial_coords_physical, ksp_values, fov_orig, img_size):
    # Convert physical k-space units back into pixel units for sigpy
    k_max_for_fov = img_size[0] / (2.0 * fov_orig *1.2788)  # Proper k_max for target FOV

    coords_normalized = radial_coords / (2 * k_max_for_fov)

    ## Flatten trajectory and ksp data
    radial_flat = coords_normalized.reshape(-1, 2) ##keep xy dimension
    ksp_flat = ksp_values.flatten()

    # Apply density compensation
    density_weights = analytical_density_weights(radial_coords_physical)
    ksp_weighted = ksp_flat * density_weights.flatten()

    
    ##Create NUFFT operator and reconstruct
    nufft_op = sp.linop.NUFFT(img_size, radial_flat)
    recon_image = nufft_op.H(ksp_weighted)
    
    return np.real(recon_image)

def crop_img(img, crop_size):
    nx, ny = img.shape
    nx_crop, ny_crop = crop_size
    center_x, center_y = nx//2, ny//2
    img_cropped = img[center_x-nx_crop//2:center_x+nx_crop//2, center_y-ny_crop//2:center_y+ny_crop//2]
    return img_cropped

def reconstruct_from_2d_cartesian(ksp_values, final_image_size):
    img = fft.fftshift(fft.fft2(fft.fftshift(ksp_values)))
    img = img /(256*256*4.987)
    cropped_img = crop_img(img, final_image_size)
    return np.abs(np.fliplr(np.flipud(cropped_img)))

def plot_reconstruction(img, motion_params,recon_type='Cartesian'):
    plt.imshow(img, cmap='gray')
    plt.title(f"{recon_type} Reconstruction with: amp = {motion_params['amplitude']}, frequency={motion_params['frequency']},\n direction = {motion_params['direction']}, scan time = {motion_params['scan_time']}")
    plt.savefig(f"recon_img_amp{motion_params['amplitude']}_freq{motion_params['frequency']}_dir{motion_params['direction']}.png")

### Parameter Sweep

In [ ]:
def run_spoke_sweep():
    '''Run parameter sweep of radial vs cartesian'''

    ## Set base params used in initial experiment
    base_params = {
    'radius': 20.0,
    'matrix_size': (256, 256),
    'matrix_size_3d': (1, 256, 256),
    'ntr': 800,  # number of spokes
    'nro': 1000, # readout points
    'ndims': 2,
    'upsamp': 2,
    'orig_size': 256
    }
    
    # Base motion parameters
    base_motion = {
        'amplitude': 2,
        'frequency': 0.02,
        'direction': 'x',
        'scan_time': 4.0
    }

    ## Set sweep parameters taken from Block et al. paper
    sweep_params = {
    'spoke_counts': [200, 400, 500, 600, 800],  # Clinical range: 400-800
    'base_resolutions': [128, 192, 256, 320],   # Clinical range: 224-384 
    'motion_amplitudes': [0, 2, 5, 10, 15, 20], # mm
    'motion_frequencies': [0.05, 0.01, 0.02, 0.05] # Hz (breathing is around 0.2-0.3 Hz)
    }
    
    spoke_sweep = []
    res_sweep = []
    amp_sweep = []
    freq_sweep = []

    ## Start sweeping
    print("Starting parameter sweep.....")
    print("------------------------------------")


    ## 1. Spoke Count/Cartesian Points
    for ntr in sweep_params['spoke_counts']:
        print(f"Testing {ntr} spokes.....")
        print("------------------------------------")

        ## Update scan time based on nTR
        motion_params = base_motion.copy()
        motion_params['scan_time'] = 0.005 * ntr  ## takes about 3-5 ms per spoke, estimate 5 ms

        ## Construct original cylinder:
        cylinder, fov_mm, _, _ = create_cylinder_with_objects_mm(
            base_params['radius'], base_params['matrix_size_3d'], add_objects=True
        )
        

        ## Get trajectories
        matrix_size = (base_params['orig_size'], base_params['orig_size'])

        ## To isolate the effect of undersampling, DO NOT include motion in radial coords
        radial_coords_pixel, radial_coords_physical = center_out_GA_radial(ntr, base_params['nro'], 
                                                                           base_params['ndims'], matrix_size, fov_mm)
        result = {
                    'ntr': ntr,
                    'cylinder_2d_mm': cylinder,
                    'scan_time': motion_params['scan_time'],
                    'motion_params': motion_params,
                    'radial_coords_pixel': radial_coords_pixel,
                    'radial_coords_physical': radial_coords_physical,
                    'fov': fov_mm
                    }
        
        spoke_sweep.append(result)

    spoke_sweep = np.array(spoke_sweep)

    return spoke_sweep
        


In [ ]:
def run_freq_sweep():
    '''Run parameter sweep of radial vs cartesian'''

    ## Set base params used in initial experiment
    base_params = {
    'radius': 20.0,
    'matrix_size': (256, 256),
    'matrix_size_3d': (1, 256, 256),
    'ntr': 800,  # number of spokes
    'nro': 1000, # readout points
    'ndims': 2,
    'upsamp': 2,
    'orig_size': 256
    }
    
    # Base motion parameters
    base_motion = {
        'amplitude': 2,
        'frequency': 0.02,
        'direction': 'x',
        'scan_time': 4.0
    }

    ## Set sweep parameters taken from Block et al. paper
    sweep_params = {
    'spoke_counts': [200, 400, 500, 600, 800],  # Clinical range: 400-800
    'base_resolutions': [128, 192, 256, 320],   # Clinical range: 224-384 
    'motion_amplitudes': [0, 2, 5, 10, 15, 20], # mm
    'motion_frequencies': [0.001, 0.01, 0.1, 0.2, 0.5] # Hz (breathing is around 0.2-0.3 Hz)
    }
    
    spoke_sweep = []
    res_sweep = []
    amp_sweep = []
    freq_sweep = []

    ## Start sweeping
    print("Starting parameter sweep.....")
    print("------------------------------------")

    cylinder, fov_mm, _, _ = create_cylinder_with_objects_mm(
            base_params['radius'], base_params['matrix_size_3d'], add_objects=True
        )
    
    radial_coords_pixel, radial_coords_physical = center_out_GA_radial(
            base_params['ntr'], base_params['nro'], base_params['ndims'], 
            base_params['matrix_size'], fov_mm
        )


    for freq in sweep_params['motion_frequencies']:
        print(f"Testing motion frequency = {freq} Hz.........")
        print("------------------------------------")

        ## Construct original cylinder:
        
        motion_params = base_motion.copy()
        motion_params['frequency'] = freq
        
        result = {'freq': freq,
                  'cylinder_2d_mm': cylinder,
                  'matrix_size': base_params['matrix_size'],
                  'motion_params': motion_params,
                'radial_coords_pixel': radial_coords_pixel,
                'radial_coords_physical': radial_coords_physical,
                  'fov': fov_mm}
        
        freq_sweep.append(result)


    freq_sweep = np.array(freq_sweep)
    return freq_sweep
        


In [ ]:
    # ## 2. Varying Resolutions
    # for res in sweep_params['base_resolutions']:
    #     print(f"Testing {res}x{res} resolution.........")
    #     print("------------------------------------")

    #     ## Set new resolution
    #     matrix_size_3d = (1, res, res)
    #     matrix_size = (res, res)
        

    #     ## Construct original cylinder:
    #     cylinder, fov_mm, _, _ = create_cylinder_with_objects_mm(
    #         base_params['radius'], base_params['matrix_size_3d'], add_objects=True
    #     )

    #     ## Get trajectories
    #     ## To isolate the effect of undersampling, DO NOT include motion in radial coords
    #     radial_coords_pixel, radial_coords_physical = center_out_GA_radial(
    #         base_params['ntr'], base_params['nro'], base_params['ndims'], 
    #         matrix_size, fov_mm
    #     )

    #     result = {'res': res,
    #               'cylinder_2d_mm': cylinder,
    #               'orig_size': res,
    #               'matrix_size': matrix_size,
    #               'motion_params': base_motion,
    #                 'radial_coords_pixel': radial_coords_pixel,
    #                 'radial_coords_physical': radial_coords_physical,
    #               'fov': fov_mm}
        
    #     res_sweep.append(res_sweep)
    
    

    # for amp in sweep_params['motion_amplitudes']:
    #     print(f"Testing motion amplitude = {amp} mm.........")
    #     print("------------------------------------")

    #     ## Construct original cylinder:
    #     cylinder, fov_mm, _, _ = create_cylinder_with_objects_mm(
    #         base_params['radius'], base_params['matrix_size_3d'], add_objects=True
    #     )
        
    #     motion_params = base_motion.copy()
    #     motion_params['amplitude'] = amp

    #     radial_coords_pixel, radial_coords_physical = center_out_GA_radial(
    #         base_params['ntr'], base_params['nro'], base_params['ndims'], 
    #         base_params['matrix_size'], fov_mm
    #     )

        
    #     result = {'amp': amp,
    #               'cylinder_2d_mm': cylinder,
    #               'matrix_size': matrix_size,
    #               'motion_params': motion_params,
    #             'radial_coords_pixel': radial_coords_pixel,
    #             'radial_coords_physical': radial_coords_physical,
    #               'fov': fov_mm}
        
    #     amp_sweep.append(amp_sweep)



    # for freq in sweep_params['motion_frequencies']:
    #     print(f"Testing motion frequency = {freq} Hz.........")
    #     print("------------------------------------")

    #     ## Construct original cylinder:
    #     cylinder, fov_mm, _, _ = create_cylinder_with_objects_mm(
    #         base_params['radius'], base_params['matrix_size_3d'], add_objects=True
    #     )
        
    #     motion_params = base_motion.copy()
    #     motion_params['frequency'] = freq

    #     radial_coords_pixel, radial_coords_physical = center_out_GA_radial(
    #         base_params['ntr'], base_params['nro'], base_params['ndims'], 
    #         base_params['matrix_size'], fov_mm
    #     )

        
    #     result = {'freq': freq,
    #               'cylinder_2d_mm': cylinder,
    #               'matrix_size': matrix_size,
    #               'motion_params': motion_params,
    #             'radial_coords_pixel': radial_coords_pixel,
    #             'radial_coords_physical': radial_coords_physical,
    #               'fov': fov_mm}
        
    #     freq_sweep.append(freq_sweep)

        

### Parameter Change Implementation

In [ ]:

# Usage:
if __name__ == "__main__":
    # Check GPU availability
    print(f"GPU available: {cp.cuda.is_available()}")
    print(f"GPU memory: {cp.get_default_memory_pool().used_bytes() / 1e9:.2f} GB")

radius = 20.
matrix_size = (256, 256)
matrix_size_3d = (1, 256, 256)
ntr = 500
nro = 1000
ndims = 2


## Set motion params to test
base_motion_params = {
                'amplitude' : 2,
                'frequency' : 0.25,
                'direction' :'x',
                'scan_time' : 4.0
                }


# Create original cylinder
cylinder_2d_mm, fov_mm, pos_mm, voxel_size_x = create_cylinder_with_objects_mm(radius, matrix_size_3d)
cylinder_orig = np.squeeze(cylinder_2d_mm)
show_2d_cylinder(cylinder_2d_mm)
# plot_motion_over_time(ntr, base_motion_params)

freq_sweep = run_freq_sweep()
print(freq_sweep.shape)


for i in range(5):
    print("Freq sweep results = ")
    print(f"    freqs = {freq_sweep[i]['freq']}")

In [ ]:
plt.figure(1)
plot_motion_over_time(800, freq_sweep[0]['motion_params'])

In [ ]:
plt.figure(2)
plot_motion_over_time(800, freq_sweep[4]['motion_params'])

### Freq Comparison

In [ ]:
freq005_radial_pixel = freq_sweep[0]['radial_coords_pixel']
freq005_radial_physical = freq_sweep[0]['radial_coords_physical']

freq05_radial_pixel = freq_sweep[4]['radial_coords_pixel']
freq05_radial_physical = freq_sweep[4]['radial_coords_physical']

freq005_motion_params = freq_sweep[0]['motion_params']
freq05_motion_params = freq_sweep[2]['motion_params']

plt.figure(1)
plot_GA_trajectory(freq005_radial_pixel)
plt.figure(2)
plot_GA_trajectory(freq05_radial_pixel)

In [ ]:
print(freq05_radial_pixel.shape)
print(freq05_radial_physical.shape)
print(analytical_density_weights(freq05_radial_physical).shape)

print(freq005_radial_pixel.shape)
print(freq005_radial_physical.shape)
print(analytical_density_weights(freq005_radial_physical).shape)

In [ ]:
# Perform Fourier encoding
S_radial_motion = fourier_encoding_radial_motion_gpu(freq005_radial_physical, cylinder_2d_mm, fov_mm, 
                                                    upsamp=2, orig_size=256, motion_params=freq005_motion_params)
S_cartesian_motion = fourier_encoding_radial_motion_gpu(freq05_radial_physical, cylinder_2d_mm, fov_mm, 
                                                    upsamp=2, orig_size=256, motion_params=freq05_motion_params)

In [ ]:
## Reconstruct with NUFFT and standard FFT
spokes200_recon_image_radial = reconstruct_from_2d_radial(freq005_radial_pixel, freq005_radial_physical, S_radial_motion, 
                                             fov_mm, matrix_size)
spokes800_recon_image_radial = reconstruct_from_2d_radial(freq05_radial_pixel, freq05_radial_physical, S_cartesian_motion, 
                                             fov_mm, matrix_size)

In [ ]:

## Visualize recon
plt.figure(1)
plot_reconstruction(spokes200_recon_image_radial, freq005_motion_params, 'Radial, freq = 0.001 Hz')
plt.figure(2)
plot_reconstruction(spokes800_recon_image_radial, freq05_motion_params, 'Radial, freq = 0.5')

In [ ]:
def calculate_image_metrics(orig, recon):
    """Calculate metrics for image quality assessment"""
    
    
    # Normalize images to same scale
    orig_norm = (orig - orig.min()) / (orig.max() - orig.min())
    recon_norm = (recon - recon.min()) / (recon.max() - recon.min())
    
    # MSE
    mse = np.mean((orig_norm - recon_norm)**2)
    rmse = np.sqrt(mse)
    
    # Peak Signal-to-Noise Ratio
    psnr = 20 * np.log10(1.0 / rmse) if rmse > 0 else float('inf')

    
    # Structural similarity (simplified)
    mean_orig = np.mean(orig_norm)
    mean_recon = np.mean(recon_norm)
    var_orig = np.var(orig_norm)
    var_recon = np.var(recon_norm)
    cov = np.mean((orig_norm - mean_orig) * (recon_norm - mean_recon))
    
    c1, c2 = 0.01**2, 0.03**2  # Constants for stability
    ssim = ((2*mean_orig*mean_recon + c1) * (2*cov + c2)) / \
           ((mean_orig**2 + mean_recon**2 + c1) * (var_orig + var_recon + c2))
    
    return {
        'mse': mse,
        'rmse': rmse, 
        'psnr': psnr,
        'ssim': ssim
    }

In [ ]:
import pandas as pd

final_metrics_cartesian = calculate_image_metrics(cylinder_2d_mm, spokes200_recon_image_radial)
final_metrics_radial = calculate_image_metrics(cylinder_2d_mm, spokes800_recon_image_radial)

df = pd.DataFrame({'Freq = 0.001': final_metrics_cartesian, 'Freq = 0.5': final_metrics_radial})
print(df)
